In [27]:
import os
import codecs
import re

import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from xclib.data import data_utils #https://github.com/kunaldahiya/pyxclib

In [28]:
save_name = "amazonCat13"
raw_data_path = "../data/raw_data/AmazonCat-13K.bow/"
save_path = "../data/amazonCat13"

In [29]:
os.listdir(raw_data_path)

['train.txt', 'Xf.txt', 'test.txt', 'Yf.txt']

In [32]:
def find(array, neq_val=None, filter_ixs=None):
    """
    params:
    array: 2D array
    neq_val: 0 for user features, labels, and None for label/item features
    """
    vals = array[array!=neq_val]
    vals = np.array(vals).flatten()
    indeces = np.argwhere(array!=neq_val)+1
    indeces_ax0 = indeces[:, 0]
    indeces_ax1 = indeces[:, 1]
    
    return vals, indeces_ax0, indeces_ax1, indeces

def save_labels(tr_points, tr_dims, te_points, te_dims, save_path, save_name, reveal_percent=""):
    np.savetxt(os.path.join(save_path, f"{save_name}_trLabelPoint{reveal_percent}.csv"), tr_points, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"{save_name}_trLabelDim{reveal_percent}.csv"), tr_dims, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"{save_name}_teLabelPoint{reveal_percent}.csv"), te_points, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"{save_name}_teLabelDim{reveal_percent}.csv"), te_dims, delimiter="\n", fmt="%-d")

def format_data(raw_data_path, save_path, save_name):
    tr_features, tr_labels, _, _, _ = data_utils.read_data(f'{raw_data_path}/train.txt')
    te_features, te_labels, _, _, _ = data_utils.read_data(f'{raw_data_path}/test.txt')
    
    tr_filter_ixs = np.array((tr_labels.sum(axis=1)>0)&(tr_features.sum(axis=1)>0)).flatten()
    tr_features = tr_features[tr_filter_ixs, :]
    tr_labels = tr_labels[tr_filter_ixs, :]
    
    te_filter_ixs = np.array((te_labels.sum(axis=1)>0)&(te_features.sum(axis=1)>0)).flatten()
    te_features = te_features[te_filter_ixs, :]
    te_labels = te_labels[te_filter_ixs, :]
    
    _, tr_y_indeces_ax0, tr_y_indeces_ax1, tr_y_indeces = find(tr_labels, 0)
    _, te_y_indeces_ax0, te_y_indeces_ax1, te_y_indeces = find(te_labels, 0)
    
    tr_x_values, tr_x_indeces_ax0, tr_x_indeces_ax1, _ = find(tr_features, 0, tr_filter_ixs)
    te_x_values, te_x_indeces_ax0, te_x_indeces_ax1, _ = find(te_features, 0, te_filter_ixs)
    
    assert set(tr_x_indeces_ax0)==set(tr_y_indeces_ax0)
    assert set(te_x_indeces_ax0)==set(te_y_indeces_ax0)
    
    np.savetxt(os.path.join(save_path, f"{save_name}_trDataValue.csv"), tr_x_values, delimiter="\n")
    np.savetxt(os.path.join(save_path, f"{save_name}_trDataPoint.csv"), tr_x_indeces_ax0, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"{save_name}_trDataDim.csv"), tr_x_indeces_ax1, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"{save_name}_teDataValue.csv"), te_x_values, delimiter="\n")
    np.savetxt(os.path.join(save_path, f"{save_name}_teDataPoint.csv"), te_x_indeces_ax0, delimiter="\n", fmt="%-d")
    np.savetxt(os.path.join(save_path, f"{save_name}_teDataDim.csv"), te_x_indeces_ax1, delimiter="\n", fmt="%-d")
    save_labels(tr_y_indeces_ax0, tr_y_indeces_ax1, te_y_indeces_ax0, te_y_indeces_ax1, save_path, save_name)

In [33]:
format_data(raw_data_path, save_path, save_name)

OSError: [Errno 122] Disk quota exceeded